# Fireside Chat AI

Ever struggled to prepare for a fireside chat or podcast with a guest speaker?

This agent automates the research and preparation process, providing you with a comprehensive briefing and insightful questions.

#### Core Features:
- Researches the guest speaker using a web search tool.
- Writes an executive summary of the guest speaker's background and expertise for the host.
- Plans a series of questions for the speaker.
- Includes source references to increase the user's [Confidence in AI Results](https://blog.langchain.com/the-hidden-metric-that-determines-ai-product-success/).

#### Future Work:
- Integrate with LangChain's [web page loaders](https://docs.langchain.com/oss/python/integrations/document_loaders#webpages) to access the entire web page content.
- Enhance the agent's ability to synthesize information from multiple sources with a more refined prompt.
- Incorporate inline citations in the executive summary and Q&A plan.
- Add support for proprietary data sources using [custom retrievers](https://docs.langchain.com/oss/python/integrations/retrievers)
- Develop evaluation metrics to assess the quality of the briefing and questions.


#### Setup
Create a `.env` file in the root directory with the following variables:

> ```sh
> OPENAI_API_KEY=your_openai_api_key
> GOOGLE_API_KEY=your_google_api_key
> GOOGLE_CSE_ID=your_google_cse_id
> ```

See the [OpenAI documentation](https://platform.openai.com/docs/api-reference/authentication) for obtaining your OpenAI API key and the [Google Custom Search documentation](https://developers.google.com/custom-search/v1/overview) for obtaining your Google API key and CSE ID.

Google provides up to 100 free search queries per day. Additional queries require a project configured with billing information.


In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from typing import Final

# The guest speaker to research
GUEST_SPEAKER: Final = "Bear Grylls"

# Helps identify the correct person
GUEST_SPEAKER_BIO: Final = "Bear Grylls is a British adventurer and television presenter."


In [ ]:
import ssl
import urllib3
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.tools import tool, Tool
from langchain_google_community import GoogleSearchAPIWrapper

# Disable SSL warnings and configure SSL handling
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

@tool(name_or_callable="google_search", description="Search Google.")
def google_search(query):
    """
    Perform a Google search for the given query.

    Args:
        query (str): The search query.
    
    Returns:
        str: The search results or a fallback message if SSL issues occur.
    """
    try:
        search_wrapper = GoogleSearchAPIWrapper()
        return search_wrapper.run(query)
    except (ssl.SSLError, Exception) as e:
        print(f"Search failed: {e}")
        return f"Search failed for query: {query}."

researcher_prompt = f"""
You are an expert podcast researcher. 

You are tasked with researching a guest speaker for a podcast episode and producing a script.

This episode's guest speaker is:
{GUEST_SPEAKER}
{GUEST_SPEAKER_BIO}

Use the Google Search tool to identify key sources of information about the guest speaker.
Output a detailed report for the podcast host, including key talking points and suggested questions for the guest speaker.
Include citations for all information sources used.
Tailor your questions so that they will be relevant and interesting to the audience: {AUDIENCE_DESCRIPTION}.

Output format

# <speaker name>

## Bio
<executive summary bio of the guest speaker, based on research. Two paragraphs.>

Key talking points you can weave into the interview:
- <talking point 1>
- <talking point 2>
- etc.

## Questions
- <question 1>
- <question 2>
- etc.

## Sources
- <source 1 name (description)>: <markdown link to source 1>
- <source 2 name (description)>: <markdown link to source 2>
- etc.

"""

research_agent = create_agent(
    model="openai:gpt-5-nano",
    tools=[google_search],
    prompt=SystemMessage(content=researcher_prompt)
)

try:
    research_agent_response = research_agent.invoke(
        {
            "messages": [
                HumanMessage(content=f"Research {GUEST_SPEAKER}"),
                AIMessage(content=f"#{GUEST_SPEAKER}\n"),
            ]
        },
    )

    print(f"Successfully researched {GUEST_SPEAKER}")
except Exception as e:
    print(f"Research failed: {e}")

In [ ]:
from IPython.display import display

display(Markdown(f"## Research Briefing for {GUEST_SPEAKER}"))
guest_speaker_briefing = research_agent_response[-1]['agent']['messages'][0].content
display(Markdown(guest_speaker_briefing))

In [ ]:
# Debug: Pretty-print the full response as JSON

import json
from IPython.display import display, Markdown

def serialize_response(obj):
    if hasattr(obj, '__dict__'):
        return {k: serialize_response(v) for k, v in obj.__dict__.items()}
    elif isinstance(obj, list):
        return [serialize_response(item) for item in obj]
    elif isinstance(obj, dict):
        return {k: serialize_response(v) for k, v in obj.items()}
    else:
        return str(obj)

serialized_response = serialize_response(research_agent_response)
formatted_json = json.dumps(serialized_response, indent=2, ensure_ascii=False)

display(Markdown(f"```json\n{formatted_json}\n```"))